<a href="https://colab.research.google.com/github/mkaramb/CloudWeaver/blob/retriever-draft/Database_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U --upgrade --quiet langchain-core langchain-community langchain

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#token = "GITHUB TOKEN"

!git clone https://{GITHUB TOKEN}:x-oauth-basic@github.com/mkaramb/CloudWeaver.git

fatal: destination path 'CloudWeaver' already exists and is not an empty directory.


In [ ]:
%cd CloudWeaver/data

/content/CloudWeaver/data


In [ ]:
base_path = '/content/CloudWeaver/data/code'

!ls "$base_path"

address		  cloud-storage   github-actions-runners  lb-http	      service-accounts
bastion-host	  composer	  group			  lb-internal	      slo
bigquery	  container-vm	  gsuite-export		  log-export	      sql-db
bootstrap	  dataflow	  healthcare		  memorystore	      startup-scripts
cloud-datastore   data-fusion	  iam			  network	      utils
cloud-dns	  datalab	  jenkins		  org-policy	      vault
cloud-nat	  event-function  kms			  pubsub	      vm
cloud-operations  folders	  kubernetes-engine	  sap		      vpc-service-controls
cloud-router	  gcloud	  lb			  scheduled-function  vpn


In [ ]:
import os

for root, dirs, files in os.walk(base_path):
    print(f"Current directory: {root}")
    print(f"Subdirectories: {dirs}")
    print(f"Files: {files}")
    # Rest of the processing logic...


In [ ]:
from langchain_core.documents import Document

def read_code_files(base_path):
    documents = []

    for root, dirs, files in os.walk(base_path):
        path_parts = root.split(os.sep)
        product, type, instance = None, None, None
        if 'code' in path_parts:
            code_index = path_parts.index('code')
            if len(path_parts) > code_index + 1:
                product = path_parts[code_index + 1]
            if len(path_parts) > code_index + 2 and path_parts[code_index + 2] in ['modules', 'examples']:
                type = path_parts[code_index + 2]
                if len(path_parts) > code_index + 3:
                    instance = path_parts[code_index + 3]

        for file in files:
            metadata = extract_metadata(file, path_parts)
            if file.endswith('.tf') or file.lower().endswith('readme.md'):
                with open(os.path.join(root, file), 'r') as content_file:
                    content = content_file.read()
                document = Document(page_content=content, metadata=metadata)
                documents.append(document)

    return documents

def extract_metadata(file_name, path_parts):
    metadata = {'type': None}
    # File type determination
    if file_name.lower() == 'main.tf':
        metadata['type'] = 'main'
    elif file_name.lower() == 'variables.tf':
        metadata['type'] = 'variables'
    elif file_name.lower() == 'outputs.tf':
        metadata['type'] = 'outputs'
    elif file_name.lower().endswith('readme.md'):
        metadata['type'] = 'readme'

    # Extract product, folder_type, instance, and sub-instance
    if 'code' in path_parts:
        code_index = path_parts.index('code')
        if len(path_parts) > code_index + 1:
            metadata['product'] = path_parts[code_index + 1]
        if len(path_parts) > code_index + 2 and path_parts[code_index + 2] in ['modules', 'examples']:
            metadata['folder_type'] = path_parts[code_index + 2]
            # Instance is the first level below modules/examples
            if len(path_parts) > code_index + 3:
                metadata['instance'] = path_parts[code_index + 3]
            # Any additional levels are considered as part of sub-instance
            if len(path_parts) > code_index + 4:
                metadata['sub_instance'] = '/'.join(path_parts[code_index + 4:])

    return metadata

base_path = '/content/CloudWeaver/data/code'
documents = read_code_files(base_path)

# Printing example metadata for demonstration
for doc in documents[:5]:  # Print the metadata of the first 5 documents
    print(doc.metadata)


{'type': 'readme', 'product': 'slo'}
{'type': 'outputs', 'product': 'slo', 'folder_type': 'examples', 'instance': 'slo-generator', 'sub_instance': 'sre_export_cloudevent_exporter'}
{'type': 'main', 'product': 'slo', 'folder_type': 'examples', 'instance': 'slo-generator', 'sub_instance': 'sre_export_cloudevent_exporter'}
{'type': 'variables', 'product': 'slo', 'folder_type': 'examples', 'instance': 'slo-generator', 'sub_instance': 'sre_export_cloudevent_exporter'}
{'type': 'readme', 'product': 'slo', 'folder_type': 'examples', 'instance': 'slo-generator', 'sub_instance': 'sre_export_cloudevent_exporter'}


In [ ]:
import pickle

with open('cw_db_metadata.pkl', 'wb') as pickle_file:
    pickle.dump(documents, pickle_file)

In [ ]:
from google.colab import files

files.download('cw_db_metadata.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>